In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires safetensors>=0.4.3, which is not installed.
transformers 4.57.3 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법with_md.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


194

In [4]:
document_list[46]

Document(metadata={'source': 'data/소득세법with_md.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준 및 세율\n\n| 종합소득 과세표준 | 세율 |\n\n|------------------|------|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009

In [5]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model = "solar-embedding-1-large"
#     model="solar-embedding-1-large-passage"
)

In [6]:
len(embedding.embed_query("소득세법"))

4096

In [8]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
# index_name = "tax-index-markdown"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
# database = PineconeVectorStore(
#     embedding=embedding,# 질문을 임베딩하여 유사도 검색
#     index_name=index_name
# )

CPU times: total: 8.08 s
Wall time: 55.4 s


# 2. 답변 생성을 위한 Retrieval

In [9]:
query = "연봉이 5천만 원 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [10]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [16]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [11]:
retrieved_docs[1]

Document(id='29675dab-956f-4d07-8ee3-68d9e690f7d0', metadata={'source': 'data/소득세법with_md.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준 및 세율\n\n| 종합소득 과세표준 | 세율 |\n\n|------------------|------|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 

# 3. 답변 생성

In [12]:
import os
from openai import OpenAI
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 5줄 이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [13]:
print(response.choices[0].message.content)

연봉 5천만 원 직장인의 근로소득세액공제는 **66만원**입니다.  
(3,300만원 초과 시 74만원에서 (5,000만원-3,300만원)×0.8% 차감 후 최소 66만원 적용)  
종합소득산출세액은 과세표준(5,000만원-공제액)에 따라 계산되며, 세율은 15%~38% 구간 적용됩니다. 정확한 세액은 추가 공제 항목(의료비, 자녀 등)을 고려해야 합니다.  

(※ 근로소득세액공제는 산출세액에서 차감되는 항목이며, 최종 세액은 종합소득산출세액 계산 후 공제 적용 필요)


In [14]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [15]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [16]:
ai_message = llm.invoke(prompt)

In [14]:
print(ai_message.content)

소득세법 및 제공된 [context]를 기반으로 연봉 5천만원인 직장인의 소득세를 계산하는 방법은 다음과 같습니다. 다만, 정확한 계산을 위해서는 근로소득공제율과 세율표 등 구체적인 수치가 필요하나, [context]에 명시된 내용을 중심으로 일반적인 절차를 설명드립니다.

---

### **1. 근로소득공제 적용 (제47조)**
- **근로소득공제**는 총급여액에서 일정 금액을 공제하는 제도입니다.  
- 공제율은 총급여액 구간에 따라 차등 적용되며, **최대 2천만원**까지 공제 가능합니다.  
- 예시 (실제 계산 시 세법 별표 참고 필요):  
  - 총급여액 5천만원일 경우, 근로소득공제액은 약 **1,200만원 ~ 1,500만원** (가칭, 정확한 수치는 세법 별표 기준)으로 추정됩니다.  
- **과세표준** = 총급여액(5,000만원) - 근로소득공제액(예: 1,500만원) = **3,500만원**

---

### **2. 종합소득산출세액 계산 (세율 적용)**
- 과세표준에 **종합소득세율**을 적용합니다 (2024년 기준 누진세율):  
  - 1,200만원 이하: 6%  
  - 1,200만원 ~ 4,600만원: 15%  
  - 4,600만원 ~ 8,800만원: 24%  
  - ... (고액 구간 생략)  
- 예시 계산:  
  - 1,200만원 × 6% = **72만원**  
  - (4,600만원 - 1,200만원) × 15% = **510만원**  
  - (3,500만원 - 4,600만원) → **해당 없음** (과세표준이 3,500만원이므로)  
  - **종합소득산출세액** = 72만원 + 510만원 = **582만원**

---

### **3. 근로소득세액공제 적용 (제59조)**
- 근로소득세액공제는 산출세액에서 추가 공제됩니다.  
- 총급여액 5,000만원인 경우:  
  - 공제액 = **74만원 - (5,000만원 - 3,300만원) × 0.08%**  
    = 74만원 - 1,400만원 × 0.0008 

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [17]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [18]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 근로소득세액공제는 총급여액 3천300만원 초과 7천만원 이하 구간에 해당하므로, 계산식은 다음과 같습니다:  \n**74만원 - [(5,000만원 - 3,300만원) × 8/1000] = 74만원 - 14.4만원 = 59.6만원**.  \n다만, 최소 공제액이 66만원 미만이면 66만원으로 적용되므로, **최종 소득세는 66만원**입니다.  \n\n(※ 단, 실제 소득세는 과세표준(공제 후 금액)과 세율을 적용한 종합소득산출세액에서 추가 공제를 거쳐 계산되며, 본 답변은 근로소득세액공제만을 고려한 것입니다.)'

# 5. 키워드 사전 활용
- 거주자/직장인, 연봉/총급여액(종합소득)

In [19]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [21]:
vec1 = embedding.embed_query("총급여")
vec2 = embedding.embed_query("연봉")
print("총급여와 연봉의 유사도: ", cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도:  0.6872200519738804


In [22]:
vec3 = embedding.embed_query("종합소득")
print("종합소득과 연봉의 유사도: ", cosine_similarity(vec3, vec2))

종합소득과 연봉의 유사도:  0.56632913237249


In [23]:
vec4 = embedding.embed_query("직장인")
vec5 = embedding.embed_query("거주자")
print("직장인과 거주자의 유사도: ", cosine_similarity(vec4, vec5))

직장인과 거주자의 유사도:  0.5402766441385678


In [24]:
query = "연봉 5천만 원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs

[Document(id='661f3bab-96ec-4947-892b-f404968df843', metadata={'source': 'data/소득세법with_md.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31., 2024. 12. 31.>\n\n1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제

In [25]:
query = "연봉 5,000만 원인 거주자의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs

[Document(id='661f3bab-96ec-4947-892b-f404968df843', metadata={'source': 'data/소득세법with_md.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31., 2024. 12. 31.>\n\n1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제

In [42]:
# 사람을 나타내는 표현 -> 거주자로 변경 / 5천만원->5000만원
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatUpstage(model="solar-pro2")
dictionary = ["사람을 나타내는 표현 → 거주자"]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 
변경해 주세요. 만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경에는 질문만 리턴해 주세요
사전 : {dictionary}
질문 : {{question}}""")

In [37]:
parser = StrOutputParser()
parser.invoke(llm.invoke(prompt.invoke({"question":"소득이 높은 남자가 있습니다"})))

'질문 : 소득이 높은 거주자가 있습니다  \n\n사전에 따라 "사람을 나타내는 표현"을 "거주자"로 변경했습니다. "5천만원"과 관련된 내용은 질문에 없으므로 변경하지 않았습니다.'

In [40]:
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke({"question":"소득이 높은 남자가 5천만원을 갖고 있습니다"})

'소득이 높은 거주자가 5000만원을 갖고 있습니다.'

In [43]:
dictionary_chain.invoke("연봉 5천만 원인 직장인의 소득세는 얼마예요?")

'연봉 5000만원인 직장인의 소득세는 얼마예요?'

In [44]:
final_chain = dictionary_chain | rag_chain

In [45]:
final_chain.invoke("연봉 5천만 원인 직장인의 소득세는 얼마예요?")

'연봉 5,000만원인 직장인의 근로소득세액공제는 문맥의 제59조 ②항 2호에 따라 계산됩니다.  \n1. 계산식: 74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = 74만원 - 1,400만원 × 0.008 = 74만원 - 11.2만원 = 62.8만원  \n2. 단, 66만원보다 적으면 66만원으로 적용되므로 최종 근로소득세액공제는 **66만원**입니다.  \n(※ 실제 소득세는 과세표준, 세율, 추가 공제 등을 종합해 계산해야 하며, 여기서는 근로소득세액공제만 답변합니다.)'

In [ ]:
# 입력 {"question": "질문"} → dictionary_chain이 질문 멘트를 수정 → 수정된 질문을 rag_chain에 전달 → retriever가 관련 문서 검색 → format_documents
# → 완성된 prompt를 llm에 전달하여 답변 생성 → 문자만 추출해서 최종 답변